# KPIs

En este apartado realizaremos la creacion de los dataframes necesarios para el calculo de cada KPIs a apartir del dataframe orginal que es el df_siniestros

## 1. Importamos librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 2. Importamos los datos originales

In [2]:
df_siniestros = pd.read_csv("Datasets\df_siniestros.csv")

# KPI 1
### Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

## 1. Creamos una nueva variable "Semestre" que basicamente divide cada año en dos partes

In [3]:
df_siniestros["Semestre"] = df_siniestros["Mes"].apply(lambda x: 1 if x <= 6 else 2)

## 2. Una vez que tenemos la columna Semestres realizamos la agrupacion por Año y Semestre para poder sumar la cantidad de homicidios que corresponde a cada uno

In [4]:
df_agrupado = df_siniestros.groupby(["Año","Semestre"]).agg({"Nro Victimas": "sum"}).reset_index()

## 3. Cuando los datos ya se encuentran agrupados creamos un dataframe final y le agregamos las columnas necesarias

In [5]:
df_kpi1 = pd.DataFrame()
df_kpi1["Año"] = df_agrupado["Año"]
df_kpi1["Semestre"] = df_agrupado["Semestre"]
df_kpi1["Homicidios Semestre Actual"] = df_agrupado["Nro Victimas"]

## 4. Calculamos una nueva variable Homicidios Semestre Anterior donde se calcula para cada semestre anterior al que se posiciona la tasa de homicidios

In [6]:
df_agrupado["Homicidios Semestre Anterior"] = df_agrupado["Nro Victimas"].shift()

## 5. Añadimos la columna Homicidios Semestre Anterior al dataframe final

In [7]:
df_kpi1["Homicidios Semestre Anterior"] = df_agrupado["Homicidios Semestre Anterior"]

## 6. Creamos una nueva variable: "Diferencia Porcentual"

In [8]:
df_kpi1["Porcentual Diferencia"] = ((df_kpi1["Homicidios Semestre Actual"] - df_kpi1["Homicidios Semestre Anterior"]) / df_kpi1["Homicidios Semestre Anterior"]) * 100

df_kpi1

,Año,Semestre,Homicidios Semestre Actual,Homicidios Semestre Anterior,Porcentual Diferencia
0,2016,1,66,NaN,NaN
1,2016,2,80,66.0,21.212121
2,2017,1,76,80.0,-5.000000
3,2017,2,79,76.0,3.947368
4,2018,1,75,79.0,-5.063291
5,2018,2,85,75.0,13.333333
6,2019,1,59,85.0,-30.588235
7,2019,2,47,59.0,-20.338983
8,2020,1,30,47.0,-36.170213
9,2020,2,56,30.0,86.666667


In [9]:
df_kpi1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           12 non-null     int64  
 1   Semestre                      12 non-null     int64  
 2   Homicidios Semestre Actual    12 non-null     int64  
 3   Homicidios Semestre Anterior  11 non-null     float64
 4   Porcentual Diferencia         11 non-null     float64
dtypes: float64(2), int64(3)
memory usage: 612.0 bytes


## 7. Cargamos el nuevo dataframe

In [31]:
df_kpi1_csv = "Datasets/df_kpi1.csv"
df_kpi1.to_csv(df_kpi1_csv, index=False)

# KPI 2
### Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

## 1. Filtramos y seleccionamos aquellos registros donde la victima es motociclista

In [12]:
df_kpi2 = df_siniestros[(df_siniestros["Victima"] == "MOTO")]

## 2. Agrupamos por Año y contamos la cantidad de homicidios

In [13]:
df_kpi2 = df_kpi2.groupby(["Año"]).size().reset_index(name="Homicidios")

## 3. Creacion de la nueva variable Año anterior donde se indica la cantidad de homicidios ocurridos el año anterior 

In [14]:
df_kpi2["Año_Anterior"] = df_kpi2["Homicidios"].shift(periods=1, fill_value=0)

## 4. Obtenemos la tasa de homicidios cada 100000 habitantes teniendo en cuenta que la poblacion de Cuidad Autonoma de Buenos Aires al censo 2022 es de 3,120,612

In [15]:
poblacion_caba = 3120612
df_kpi2["Tasa_Homicidios"] = (df_kpi2["Homicidios"] / poblacion_caba) * 100000
df_kpi2["Tasa_Año_Anterior"] = (df_kpi2["Año_Anterior"] / poblacion_caba) * 100000

## 5. Creacion de la variable Variacion Porcentual

In [16]:
df_kpi2["Variacion"] = ((df_kpi2["Tasa_Homicidios"] - df_kpi2["Tasa_Año_Anterior"]) / df_kpi2["Tasa_Año_Anterior"]) * 100
df_kpi2["Variacion"] = df_kpi2["Variacion"].replace([np.inf, -np.inf], np.nan)
df_kpi2

,Año,Homicidios,Año_Anterior,Tasa_Homicidios,Tasa_Año_Anterior,Variacion
0,2016,63,0,2.018835,0.000000,NaN
1,2017,54,63,1.730430,2.018835,-14.285714
2,2018,57,54,1.826565,1.730430,5.555556
3,2019,50,57,1.602250,1.826565,-12.280702
4,2020,27,50,0.865215,1.602250,-46.000000
5,2021,45,27,1.442025,0.865215,66.666667


## 6. Cargamos el nuevo dataframe

In [17]:
df_kpi2_csv = "Datasets/df_kpi2.csv"
df_kpi2.to_csv(df_kpi2_csv, index=False)